In [1]:
import numpy as np
import random
import pandas as pd

from sklearn.model_selection import train_test_split

from IPython.display import Audio, display
def allDone():
    urL = 'https://www.wavsource.com/snds_2020-10-01_3728627494378403/animals/cat_meow2.wav'
    display(Audio(url=urL, autoplay=True))
# allDone()

random.seed(666)

In [2]:
# motifs = np.genfromtxt('data_dev/',dtype='U')
# motifxFamMatrix = np.genfromtxt('data_dev/motifxFamMatrix.csv',delimiter=',',dtype=int)
# fams = np.genfromtxt('data_dev/fams.csv',dtype='U')

# print(len(motifs))

# X_train, X_test = train_test_split(range(len(motifs)), test_size=0.15, random_state=666)

# train_motifs = motifs[X_train]
# test_motifs = motifs[X_test]

# len(train_motifs)

In [6]:
# Try original data from EMBER 1.
train_motifs = np.genfromtxt('data_dev/train_motifs_unfiltered.csv',dtype='U')
test_motifs = np.genfromtxt('data_dev/test_motifs_unfiltered.csv',dtype='U')
train_motifxFamMatrix = np.genfromtxt('data_dev/train_motifxFamMatrix_unfiltered.csv',delimiter=',',dtype=int)
test_motifxFamMatrix = np.genfromtxt('data_dev/test_motifxFamMatrix_unfiltered.csv',delimiter=',',dtype=int)

print(train_motifs.shape)
print(test_motifs.shape)

train_motifs = np.genfromtxt('data_dev/train_motifs.csv',dtype='U')
print(train_motifs.shape)

train_motifs = np.genfromtxt('data_dev/new_test_motifs.csv',dtype='U')
print(train_motifs.shape)


(6678,)
(857,)
(6449,)
(853,)


In [18]:
6678-6449

229

## Look at CD-HIT results.

In [4]:
# from Bio import SeqIO

# results = '1588970924.result/1588970924.fas.db2novel.clstr.sorted'

# fasta_sequences = SeqIO.parse(open(results),'fasta')
# for i,fasta in enumerate(fasta_sequences):
#     name, sequence = fasta.id, str(fasta.seq)
# #     print(i,sequence.split())
#     if i==1000:
#         break
        
# m1 = train_motifs[2268]
# m2 = test_motifs[128]

# print(m1, m2, m1[7], m2[7])

# i1 = np.where(motifs==m1)[0][0]
# i2 = np.where(motifs==m2)[0][0]

# print(i1,i2)

# fi1 = np.where(motifxFamMatrix[i1]==1)[0]
# fi2 = np.where(motifxFamMatrix[i2]==1)[0]
# print(fi1)
# print(fi2)

# print(motifxFamMatrix[i1])
# print(motifxFamMatrix[i2])

# print(fams[fi1])
# print(fams[fi2])

## Find Hamming-near motifs between test and train sets.

In [18]:
### https://biology.stackexchange.com/questions/23523/hamming-distance-between-two-dna-strings
def inv_hamming_distance_COPYPASTA(s1, s2):
    #Return the Hamming distance between equal-length sequences
    if len(s1) != len(s2):
        raise ValueError("Undefined for sequences of unequal length")
    return sum(ch1 == ch2 for ch1, ch2 in zip(s1, s2)) / len(s1)

In [19]:
def inv_hamming_distance(s1, s2):
    #Return the Hamming distance between equal-length sequences
    if len(s1) != len(s2):
        raise ValueError("Undefined for sequences of unequal length")
    zipped = zip(s1,s2)
    ### don't want to count "empty" characters ...
    max_buff_length = max( sum([1 for c in s1 if c=='_']), sum([1 for c in s2 if c=='_'])  )
    return (sum((ch1 == ch2 and ch1 != '_' and ch2 != '_') for ch1, ch2 in zip(s1, s2)) / 
            (len(s2)-max_buff_length) )

In [20]:
t1 = 'GEDEESESD______'
t2 = 'AEEKEAKSD______'

print(inv_hamming_distance(t1,t2))
print(inv_hamming_distance_COPYPASTA(t1,t2))

print(inv_hamming_distance_COPYPASTA('GEDEESESD','AEEKEAKSD'))
print(inv_hamming_distance('GEDEESESD','AEEKEAKSD'))

0.4444444444444444
0.6666666666666666
0.4444444444444444
0.4444444444444444


In [21]:
import time
import itertools

start = time.time()
similar_motifs = []
for i,combo in enumerate(itertools.product(train_motifs, test_motifs)):
    score = inv_hamming_distance(combo[0],combo[1])
    if score >= 0.6:
#         print(score, combo)
        similar_motifs.append( [combo[0], combo[1], score] )
print("%5.3f mins" % ((time.time()-start)/60))
allDone()


0.434 mins


In [22]:
sim_train = list(set([x[0] for x in similar_motifs]))
sim_test = list(set([x[1] for x in similar_motifs]))

print(len(sim_train))
print(len(sim_test))

229
184


## Define and save training set indices to remove ... 

In [24]:
idc_to_remove = []
for motif in sim_train:
    idx = np.where(train_motifs==motif)[0][0]
    idc_to_remove.append(idx)
idc_to_remove = np.array((idc_to_remove))
print(idc_to_remove)

idc_to_keep = [x for x in range(len(train_motifs)) if x not in idc_to_remove]
X_train = idc_to_keep

print(len(X_train))

train_motifs = train_motifs[X_train]
train_motifxFamMatrix = train_motifxFamMatrix[X_train]

data_dir = "data_old/"

df = pd.DataFrame(train_motifs,dtype='U')
df.to_csv(data_dir + 'train_motifs.csv',header=None,index=None)
df = pd.DataFrame(test_motifs,dtype='U')
df.to_csv(data_dir + 'test_motifs.csv',header=None,index=None)

df = pd.DataFrame(train_motifxFamMatrix,dtype=int)
df.to_csv(data_dir + 'train_motifxFamMatrix.csv',header=None,index=None)
df = pd.DataFrame(test_motifxFamMatrix,dtype=int)
df.to_csv(data_dir + 'test_motifxFamMatrix.csv',header=None,index=None)

[4639 1837 4602   57 2806 5726 2123 5493 3513 5524 4068 2173 2990 1206
 1154 5811 6284 4454 1235 1612  581 2310 5243 2620 5361 3668 2400 3060
 1852 4975  623 3577 6306  230 3437 3454 1752 3249 4196 4685  332 4765
 5294 5365 4467  127 4365  634 3976 4705 1546 1878  212 4270 3460 3114
  180 4750 5252 5891 3409 5282 4280 1460 3835 4204 1595 1513 3737 4395
 5257 2621  252 4078 3647 1874 1156 3083 3868 2743 1575 5608 3769 3574
 1322 4669 4269 6574 3480 5338 5358 4607 5222 5276 1093 2261 6117 1317
 5842  407 2349 3732 2527   75 3017  798  251  424 2282 5496 5592 4159
 4697 6250 2339 3785 3404 3780  476 4683 1646 3700 6177 3046 6441  799
  875 4356  281 5371 5662 6657  107 4465 2103 1511 1538 5140 5758 1228
 3255 4340  266 6105 3833 5661 5969   97  755 4784 1205 3507 4699 1472
 1905 3476 2934 3376 3247 6494 4491 1108 2111 2951  727 3080 1506 4859
 6239 1428 4399 5525 6659 2723  313 4024 5570 5964  475 4305 3326 4319
 3617 6437 4524 5653 1883  935  988 2502 5203 1426 3992 5413  737 2554
 4963 

In [28]:
print(len(test_motifs))
print(len(train_motifs))

print(len(train_motifs)+len(test_motifs))

857
6449
7306


In [ ]:
# idc_to_remove = []
# for motif in sim_train:
#     idx = np.where(motifs==motif)[0][0]
#     idc_to_remove.append(idx)
# idc_to_remove = np.array((idc_to_remove))
# print(idc_to_remove)

# idc_to_keep = [x for x in X_train if x not in idc_to_remove]
# X_train = idc_to_keep

# print(len(X_train))

# train_motifs = motifs[X_train]
# test_motifs = motifs[X_test]

# train_motifxFamMatrix = motifxFamMatrix[X_train]
# test_motifxFamMatrix = motifxFamMatrix[X_test]

# data_dir = "data_old/"

# df = pd.DataFrame(train_motifs,dtype='U')
# df.to_csv(data_dir + 'train_motifs.csv',header=None,index=None)
# df = pd.DataFrame(test_motifs,dtype='U')
# df.to_csv(data_dir + 'test_motifs.csv',header=None,index=None)

# df = pd.DataFrame(train_motifxFamMatrix,dtype=int)
# df.to_csv(data_dir + 'train_motifxFamMatrix.csv',header=None,index=None)
# df = pd.DataFrame(test_motifxFamMatrix,dtype=int)
# df.to_csv(data_dir + 'test_motifxFamMatrix.csv',header=None,index=None)

## Getting rid of motifs that I can't find on uniprot (notebook 06). (Need to be able to find all motifs so that I can elongate them for musite and deepPhos comparison.)

In [29]:
train_motifs = np.genfromtxt('data_dev/old_test_and_train/train_motifs.csv',dtype='U')
train_motifxFamMatrix = np.genfromtxt('data_dev/old_test_and_train/train_motifxFamMatrix.csv',delimiter=',',dtype=int)
test_motifs = np.genfromtxt('data_dev/old_test_and_train/test_motifs.csv',dtype='U')
test_motifxFamMatrix = np.genfromtxt('data_dev/old_test_and_train/test_motifxFamMatrix.csv',delimiter=',',dtype=int)

fams = np.genfromtxt('data_dev/fams.csv',dtype='U')

all_motifs = np.hstack([train_motifs,test_motifs])
all_motifxFamMatrix = np.vstack([train_motifxFamMatrix,test_motifxFamMatrix])

In [31]:
# bad_motifs = ['AIVHLINYQDDAELV', '_PKRKVSSAEGAAKE', 'RCCDEVMSKDPTHWA', 'GQLIGLNSPNSISRA', 
#                'HSTPASAYGSVKAYT', 'QAAYYGQTPVPGPQP', 'QSVTRATSICGSGVK', 'ESRPRAVSFRQSEPS', 
#                '__MDEDYYGSAAEWG', 'ARLKSPLSTLDWTVI' ]

bad_motifs = ['_PKRKVSSAEGAAKE','ISQVRRRSSLLGSQL','ESRPRAVSFRQSEPS','LGPGPRLSLARMPPP']

In [32]:
bad_idc = [ np.where(test_motifs==m)[0][0] for m in bad_motifs ]

In [33]:
good_idc = [x for x in range(len(test_motifs)) if x not in bad_idc]
new_test_motifs = test_motifs[good_idc]
new_test_motifxFamMatrix = test_motifxFamMatrix[good_idc]

print(len(test_motifs))
print(len(new_test_motifs))

data_dir = "data_dev/old_test_and_train/"

df = pd.DataFrame(new_test_motifs,dtype='U')
df.to_csv(data_dir + 'new_test_motifs.csv',header=None,index=None)
df = pd.DataFrame(new_test_motifxFamMatrix,dtype='U')
df.to_csv(data_dir + 'new_test_motifxFamMatrix.csv',header=None,index=None)

857
853


## Meta-data

In [35]:
print("train motifs:",len(train_motifs))
print("test motifs:",len(new_test_motifs))
print("total motifs:",len(new_test_motifs)+len(train_motifs))

train motifs: 6449
test motifs: 853
total motifs: 7302


In [12]:
fams = np.genfromtxt('data_dev/fams.csv',dtype='U')

In [13]:
train_motifs = np.genfromtxt('data_dev/train_motifs.csv',dtype='U')
train_motifxFamMatrix = np.genfromtxt('data_dev/train_motifxFamMatrix.csv',delimiter=',',dtype=int)
test_motifs = np.genfromtxt('data_dev/new_test_motifs.csv',dtype='U')
test_motifxFamMatrix = np.genfromtxt('data_dev/new_test_motifxFamMatrix.csv',delimiter=',',dtype=int)

In [14]:
fams_alphOrder = ['AKT','CDK','CK2',"MAPK","PIKK", "PKA","PKC","SRC"]

print("Train motifs | test motifs ... per fam")
for fam in fams_alphOrder:
    f_idx = np.where(fams==fam)[0][0]
    print(fam,sum(train_motifxFamMatrix[:,f_idx]),
         "|",sum(test_motifxFamMatrix[:,f_idx]))

Train motifs | test motifs ... per fam
AKT 382 | 63
CDK 752 | 116
CK2 775 | 97
MAPK 1275 | 187
PIKK 497 | 63
PKA 1235 | 231
PKC 1497 | 251
SRC 869 | 99
